In [4]:
import mne
import os
from tqdm import tqdm_notebook
import neurokit2 as nk
import numpy as np
import yasa
from scipy.signal import welch

In [5]:
import warnings
import logging
import pandas as pd
warnings.filterwarnings('ignore')

logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
logging.getLogger("mne").setLevel(logging.WARNING)

In [6]:
pd.read_excel('ratings/Result movies.xlsx')

,Participant,a.mp4,b.mp4,c.mp4,d.mp4,e.mp4,f.mp4,g.mp4,h.mp4,Unnamed: 9,порядок предъявления,Unnamed: 11,Unnamed: 12
0,1,10,10.000000,2.000000,5.000000,9.000000,9.000000,5.000000,9.000000,NaN,14853762,NaN,NaN
1,2,10,8.000000,7.000000,9.000000,7.000000,6.000000,6.000000,9.000000,NaN,26753841,NaN,NaN
2,3,6,3.000000,5.000000,6.000000,5.000000,5.000000,5.000000,2.000000,NaN,83261475,NaN,NaN
3,4,8,6.000000,5.000000,9.000000,5.000000,7.000000,4.000000,6.000000,NaN,57164283,NaN,NaN
4,5,3,2.000000,4.000000,1.000000,7.000000,3.000000,7.000000,2.000000,NaN,72481536,NaN,NaN
5,6,7,7.000000,2.000000,8.000000,5.000000,3.000000,2.000000,5.000000,NaN,14853762,NaN,NaN
6,7,9,4.000000,4.000000,3.000000,9.000000,8.000000,8.000000,5.000000,NaN,26753841,NaN,NaN
7,8,5,7.000000,5.000000,9.000000,3.000000,2.000000,8.000000,7.000000,NaN,83261475,NaN,NaN
8,9,8,8.000000,2.000000,3.000000,7.000000,5.000000,5.000000,9.000000,NaN,57164283,NaN,NaN
9,10,3,8.000000,7.000000,3.000000,1.000000,3.000000,6.000000,2.000000,NaN,72481536,NaN,NaN


In [7]:
ratings = pd.read_excel('ratings/Result movies.xlsx')
ratings = ratings[0 : 21][['a.mp4', 'b.mp4', 'c.mp4', 'd.mp4', 'e.mp4', 'f.mp4','g.mp4', 'h.mp4']]
ratings

,a.mp4,b.mp4,c.mp4,d.mp4,e.mp4,f.mp4,g.mp4,h.mp4
0,10,10.0,2.0,5.0,9.0,9.0,5.0,9.0
1,10,8.0,7.0,9.0,7.0,6.0,6.0,9.0
2,6,3.0,5.0,6.0,5.0,5.0,5.0,2.0
3,8,6.0,5.0,9.0,5.0,7.0,4.0,6.0
4,3,2.0,4.0,1.0,7.0,3.0,7.0,2.0
5,7,7.0,2.0,8.0,5.0,3.0,2.0,5.0
6,9,4.0,4.0,3.0,9.0,8.0,8.0,5.0
7,5,7.0,5.0,9.0,3.0,2.0,8.0,7.0
8,8,8.0,2.0,3.0,7.0,5.0,5.0,9.0
9,3,8.0,7.0,3.0,1.0,3.0,6.0,2.0


In [5]:
watch = mne.io.read_raw('raw_full/021.vhdr')

In [6]:
filtered = mne.io.read_raw('raw_filtered_fif/S01_filtered_raw.fif')
raw = watch = mne.io.read_raw('raw_full/001.vhdr')
annot = filtered.annotations
annot[0]

OrderedDict([('onset', 2.295),
             ('duration', 20.0),
             ('description', 'relax_before_a'),
             ('orig_time', None)])

In [7]:
raw = raw.crop(annot[0]['onset'], annot[0]['onset'] + annot[0]['duration'])

In [116]:
cleaned = nk.ecg_clean(raw['Pulse'][0][0], sampling_rate=1000)
info = nk.ecg_findpeaks(cleaned)
np.mean(nk.ecg_rate(info['ECG_R_Peaks'], 1000))

68.66879025662902

In [12]:
n_subjects = 21
n_films = 8
n_channels = 18
channel_names = ['Fz', 'F3', 'F7','C3', 'T7', 'Pz', 'P3','P7', 'O1', 'Oz', 'O2','P4', 'P8', 'Cz','C4', 'T8', 'F4', 'F8']
for Subject in tqdm_notebook(range(4, 5)):
    if Subject == 5 or Subject == 21:
        continue
    for film in ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']:
        if not os.path.isfile('cropped_new/{}/watch_{}.fif'.format(Subject, film)):
            continue

        # читаем просмотр из файла
        
        raw = mne.io.read_raw('cropped_new/{}/watch_{}.fif'.format(Subject, film))
        
        #удаляем ненужные каналы
        
        if 'VEOG' in raw.ch_names:
            raw.drop_channels(['VEOG'])
        raw.drop_channels(['Pulse', 'Zygom', 'Corr', 'EDA', 'Mark'])
        
        for channel in range(n_channels):
#            if os.path.isfile('features_neurokit/S_{}_F_{}_{}_.xlsx'.format(Subject, film, channel_names[channel])):
#                continue
            df, info = nk.complexity(raw[channel_names[channel]][0][0], which = ['fast'])
            df['Subj'] =  Subject
            df['film'] = ord(film) - 96
            df['ch'] = channel + 1
            df['labels'] = ratings.loc[Subject - 1][film + '.mp4']
            df.to_excel('features_neurokit_2/S_{}_F_{}_{}_.xlsx'.format(Subject, film, channel_names[channel]), index=False)

  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
import mne_features

In [13]:
n_subjects = 21
n_films = 8
n_channels = 18
channel_names = ['Fz', 'F3', 'F7','C3', 'T7', 'Pz', 'P3','P7', 'O1', 'Oz', 'O2','P4', 'P8', 'Cz','C4', 'T8', 'F4', 'F8']
for Subject in tqdm_notebook(range(1, n_subjects + 1)):
    if Subject == 5 or Subject == 21:
        continue
    for film in ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']:
        if not os.path.isfile('cropped_new/{}/watch_{}.fif'.format(Subject, film)):
            continue
        raw = mne.io.read_raw('cropped_new/{}/relax_{}.fif'.format(Subject, film))
        if 'VEOG' in raw.ch_names:
            raw.drop_channels(['VEOG'])
        if 'STI' in raw.ch_names:
            raw.drop_channels(['STI'])
        raw.drop_channels(['Pulse', 'Zygom', 'Corr', 'EDA', 'Mark'])

        data = raw.get_data()
        data = data.reshape(1, data.shape[0] , data.shape[1])
        if not os.path.isfile('Features_mnef/S{}F{}biv_relax.xlsx'.format(Subject, film)):
            funcs_bi = mne_features.bivariate.get_bivariate_funcs(raw.info['sfreq'])
            funcs_bi.pop('nonlin_interdep')
            X_bi = mne_features.feature_extraction.extract_features(data, raw.info['sfreq'], funcs_bi, ch_names=raw.ch_names, n_jobs=-1, return_as_df=True)
            X_bi.to_excel('Features_mnef/S{}F{}biv_relax.xlsx'.format(Subject, film))
        if not os.path.isfile('Features_mnef/S{}F{}univ_relax.xlsx'.format(Subject, film)):
            funcs = mne_features.univariate.get_univariate_funcs(raw.info['sfreq'])
            funcs.pop('svd_entropy')
            funcs.pop('app_entropy')
            funcs.pop('hjorth_complexity')
            funcs.pop('svd_fisher_info')
            funcs.pop('hurst_exp')
            funcs.pop('pow_freq_bands')
            funcs.pop('katz_fd')
            funcs.pop('spect_entropy')
            funcs.pop('samp_entropy')
            X = mne_features.feature_extraction.extract_features(data, raw.info['sfreq'], funcs, n_jobs=-1,ch_names=raw.ch_names, return_as_df=True)
            X.to_excel('Features_mnef/S{}F{}univ_relax.xlsx'.format(Subject, film))

  0%|          | 0/21 [00:00<?, ?it/s]

In [9]:
result = pd.DataFrame()
ind_ex = 0
temp = pd.DataFrame()
n_subjects = 21
n_films = 8
n_channels = 18
chans = ['Fz', 'F3', 'F7','C3', 'T7', 'Pz', 'P3','P7', 'O1', 'Oz', 'O2','P4', 'P8', 'Cz','C4', 'T8', 'F4', 'F8']
for Subject in tqdm_notebook(range(1, n_subjects + 1)):
    if Subject == 5 or Subject == 21:
        continue
    for film in ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']:
        if not os.path.isfile('cropped_new/{}/watch_{}.fif'.format(Subject, film)):
            continue
        conc = pd.DataFrame()

        if not os.path.isfile('Features_mnef/S{}F{}biv.xlsx'.format(Subject, film)):
            continue
        biv = pd.read_excel('Features_mnef/S{}F{}biv.xlsx'.format(Subject, film))
        univ = pd.read_excel('Features_mnef/S{}F{}univ.xlsx'.format(Subject, film))
        columns = [elem for elem in univ.columns if elem.split(':')[0] != 'Unnamed']
        columns_biv = [elem for elem in biv.columns if elem.split(':')[0] != 'Unnamed']
        temp_univ = pd.DataFrame()
        temp_biv = pd.DataFrame()
        for k in range(0, len(columns)):
            current_univ = columns[k]
            if k == len(columns) - 1:
                values_univ = univ.loc[2:, current_univ:].values[0]
                columns_univ = np.array(univ.loc[0:,  current_univ].dropna())[0]
            else:
                nxt_unive = columns[k + 1]
                values_univ = univ.loc[2:, current_univ : nxt_unive].drop([nxt_unive], axis=1).values
                columns_univ = np.array(univ.loc[0:,  current_univ : nxt_unive].drop([nxt_unive], axis=1).dropna())[0]
            if current_univ == 'pow_freq_bands':
                continue
            new_columns_univ = []
            if len(columns_univ) > 18:
                for l in range(len(columns_univ)):
                    if current_univ == 'teager_kaiser_energy':
                        new_columns_univ.append(current_univ + '_' + ''.join(columns_univ[l].split('_')[1:]))
                    else:
                        new_columns_univ.append(current_univ + '_' + columns_univ[l].split('_')[-1])
            df_univ = pd.DataFrame(values_univ.reshape((18, -1)), columns=list(dict.fromkeys(new_columns_univ)) if len(columns_univ) > 18 else [current_univ], index=[i for i in range(ind_ex, ind_ex + 18)])
            temp_univ = pd.concat([temp_univ, df_univ], axis=1)
            
        for p in range(0, len(columns_biv)):
            current_biv = columns_biv[p]
            if p == len(columns_biv) - 1:
                values_biv = biv.loc[2:, current_biv:].values[0]
                columns = np.array(biv.loc[0:,  current_biv].dropna())[0]
            else:
                nxt_biv = columns_biv[p + 1]
                values_biv = biv.loc[2:, current_biv : ].drop([nxt_biv], axis=1).values[0]
                columns = np.array(biv.loc[0:,  current_biv : ].drop([nxt_biv], axis=1).dropna())[0]
            new_columns_biv = []
            for l in range(len(chans)):
                new_columns_biv.append(current_biv + '_' + chans[l])
            matrix = np.zeros((18, 18))
            left = 0
            ind = 17
            for d in range(0, n_channels):
                for k in range(d, n_channels):
                    if  d != k:
                        matrix[d][k] = values_biv[left:ind][k - d - 1]
                        matrix[k][d] = values_biv[left:ind][k - d - 1]
                left = ind
                ind += 17 - d - 1

            df_biv = pd.DataFrame(matrix, columns=new_columns_biv, index=[i for i in range(ind_ex, ind_ex + 18)])
            if current_biv == 'time_corr':
                df_eig = pd.DataFrame(values_biv[ind + 1:len(values_biv)], columns=['eig'], index=[i for i in range(ind_ex, ind_ex + 18)])
                df_biv = pd.concat([df_biv, df_eig], axis=1)
            temp_biv = pd.concat([temp_biv, df_biv], axis=1)
        conc = pd.concat([temp_univ, temp_biv], axis=1)
        conc['film'] = film 
        conc['labels'] = ratings.loc[Subject - 1][film + '.mp4']
        conc['Subj'] = Subject
        conc['ch'] = chans[n_chan]
        n_chan += 1 
        result = pd.concat([result, conc])
        ind_ex += 18
print(result.shape)

  0%|          | 0/21 [00:00<?, ?it/s]

(2700, 119)


In [27]:
result

,decorr_time,energy_freq_bands_band0,energy_freq_bands_band1,energy_freq_bands_band2,higuchi_fd,hjorth_complexity_spect,hjorth_mobility,hjorth_mobility_spect,kurtosis,line_length,...,time_corr_P8,time_corr_Cz,time_corr_C4,time_corr_T8,time_corr_F4,time_corr_F8,eig,film,labels,Subj
0,-1,0.0,0.0,0.0,1.51975,0.000001,0.275248,0.0,3.50806,0.000002,...,-0.338355,0.665202,0.440300,-0.599733,0.473322,-0.586361,0.0,a,10.0,1
1,-1,0.0,0.0,0.0,1.613473,0.000002,0.329742,0.0,3.239525,0.000002,...,-0.593253,0.305272,-0.079909,-0.623630,-0.267212,-0.447630,0.0,a,10.0,1
2,-1,0.0,0.0,0.000001,1.674659,0.000006,0.293204,0.0,3.051866,0.000003,...,-0.294019,-0.262123,-0.607796,-0.058566,-0.422053,0.298100,0.0,a,10.0,1
3,-1,0.0,0.0,0.000001,1.688218,0.000003,0.336888,0.0,2.958401,0.000002,...,-0.294201,0.310086,0.390273,-0.323726,0.029401,-0.503890,0.0,a,10.0,1
4,-1,0.0,0.0,0.000001,1.709531,0.000007,0.410434,0.0,3.100358,0.000003,...,-0.205999,-0.422331,-0.518193,-0.079592,-0.380073,0.130950,0.0,a,10.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,-1,0.0,0.0,0.000001,1.570978,0.000002,0.179934,0.0,9.176377,0.000002,...,-0.381015,0.000000,0.296897,-0.208100,-0.045185,-0.152193,1.272184,h,6.0,20
2696,-1,0.0,0.0,0.000001,1.78204,0.000007,0.297171,0.0,8.4125,0.000003,...,0.008490,0.296897,0.000000,0.102345,0.523751,-0.140699,1.782883,h,6.0,20
2697,-1,0.0,0.0,0.000001,1.673019,0.000004,0.298448,0.0,3.883632,0.000002,...,0.521226,-0.208100,0.102345,0.000000,0.108494,0.660035,3.140469,h,6.0,20
2698,-1,0.0,0.0,0.000002,1.887136,0.000008,0.516783,0.0,3.034146,0.000003,...,-0.200788,-0.045185,0.523751,0.108494,0.000000,0.144154,3.688495,h,6.0,20


In [28]:
mnef_watch = pd.read_csv('Matrix_mnef.csv')
mnef_watch

,decorr_time,energy_freq_bands_band0,energy_freq_bands_band1,energy_freq_bands_band2,higuchi_fd,hjorth_complexity_spect,hjorth_mobility,hjorth_mobility_spect,kurtosis,line_length,...,time_corr_P8,time_corr_Cz,time_corr_C4,time_corr_T8,time_corr_F4,time_corr_F8,eig,film,labels,Subj
0,-1.0,1.058385e-07,1.397773e-07,3.278018e-07,1.519750,0.000001,0.275248,3.587811e-09,3.508060,0.000002,...,-0.338355,0.665202,0.440300,-0.599733,0.473322,-0.586361,2.843967e-16,a,10.0,1
1,-1.0,6.522955e-08,1.217258e-07,3.346958e-07,1.613473,0.000002,0.329742,3.393147e-09,3.239525,0.000002,...,-0.593253,0.305272,-0.079909,-0.623630,-0.267212,-0.447630,1.141450e-11,a,10.0,1
2,-1.0,1.603566e-07,2.581948e-07,1.146924e-06,1.674659,0.000006,0.293204,1.161045e-08,3.051866,0.000003,...,-0.294019,-0.262123,-0.607796,-0.058566,-0.422053,0.298100,3.459969e-11,a,10.0,1
3,-1.0,6.296471e-08,1.256659e-07,5.082828e-07,1.688218,0.000003,0.336888,4.968475e-09,2.958401,0.000002,...,-0.294201,0.310086,0.390273,-0.323726,0.029401,-0.503890,5.284086e-11,a,10.0,1
4,-1.0,1.348794e-07,2.556185e-07,1.272476e-06,1.709531,0.000007,0.410434,1.265314e-08,3.100358,0.000003,...,-0.205999,-0.422331,-0.518193,-0.079592,-0.380073,0.130950,1.105781e-10,a,10.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,-1.0,1.198905e-07,1.962642e-07,5.534923e-07,1.570978,0.000002,0.179934,4.407703e-09,9.176377,0.000002,...,-0.381015,0.000000,0.296897,-0.208100,-0.045185,-0.152193,1.272184e+00,h,6.0,20
2696,-1.0,8.276883e-08,1.388573e-07,1.346255e-06,1.782040,0.000007,0.297171,1.069583e-08,8.412500,0.000003,...,0.008490,0.296897,0.000000,0.102345,0.523751,-0.140699,1.782883e+00,h,6.0,20
2697,-1.0,1.159373e-07,1.994772e-07,1.033203e-06,1.673019,0.000004,0.298448,8.067409e-09,3.883632,0.000002,...,0.521226,-0.208100,0.102345,0.000000,0.108494,0.660035,3.140469e+00,h,6.0,20
2698,-1.0,3.440238e-08,9.217395e-08,1.591280e-06,1.887136,0.000008,0.516783,1.256575e-08,3.034146,0.000003,...,-0.200788,-0.045185,0.523751,0.108494,0.000000,0.144154,3.688495e+00,h,6.0,20


In [29]:
fls = mnef_watch[['film', 'labels', 'Subj']]
fls

,film,labels,Subj
0,a,10.0,1
1,a,10.0,1
2,a,10.0,1
3,a,10.0,1
4,a,10.0,1
...,...,...,...
2695,h,6.0,20
2696,h,6.0,20
2697,h,6.0,20
2698,h,6.0,20


In [33]:
res = mnef_watch.drop(['film', 'labels', 'Subj'], axis=1).sub(result.drop(['film', 'labels', 'Subj'], axis=1), axis='columns')
res

,decorr_time,energy_freq_bands_band0,energy_freq_bands_band1,energy_freq_bands_band2,higuchi_fd,hjorth_complexity_spect,hjorth_mobility,hjorth_mobility_spect,kurtosis,line_length,...,time_corr_Oz,time_corr_O2,time_corr_P4,time_corr_P8,time_corr_Cz,time_corr_C4,time_corr_T8,time_corr_F4,time_corr_F8,eig
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,1.387779e-17,0.000000e+00,0.000000e+00,0.0,-0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.979728e-17,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.326673e-17,0.000000e+00,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-4.163336e-17,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.0,6.938894e-17,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.551115e-17,0.0,0.0
2696,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.0,0.000000e+00,-4.683753e-17,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0
2697,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.0,8.326673e-17,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0
2698,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.994932e-17,0.0,4.857226e-17,0.000000e+00,5.551115e-17,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0


In [25]:
result.to_excel('Matrix_mnef_relax.xlsx', index=False)

In [22]:
result.to_csv('Matrix_mnef_relax.csv', index=False)

In [18]:
result.to_csv('Matrix_mnef_relax.csv', index=False)

In [26]:
mnef_relax = pd.read_excel('Matrix_mnef_relax.xlsx')
mnef_relax

,decorr_time,energy_freq_bands_band0,energy_freq_bands_band1,energy_freq_bands_band2,higuchi_fd,hjorth_complexity_spect,hjorth_mobility,hjorth_mobility_spect,kurtosis,line_length,...,time_corr_P8,time_corr_Cz,time_corr_C4,time_corr_T8,time_corr_F4,time_corr_F8,eig,film,labels,Subj
0,-1.0,1.058385e-07,1.397773e-07,3.278018e-07,1.519750,0.000001,0.275248,3.587811e-09,3.508060,0.000002,...,-0.338355,0.665202,0.440300,-0.599733,0.473322,-0.586361,2.843967e-16,a,10,1
1,-1.0,6.522955e-08,1.217258e-07,3.346958e-07,1.613473,0.000002,0.329742,3.393147e-09,3.239525,0.000002,...,-0.593253,0.305272,-0.079909,-0.623630,-0.267212,-0.447630,1.141450e-11,a,10,1
2,-1.0,1.603566e-07,2.581948e-07,1.146924e-06,1.674659,0.000006,0.293204,1.161045e-08,3.051866,0.000003,...,-0.294019,-0.262123,-0.607796,-0.058566,-0.422053,0.298100,3.459969e-11,a,10,1
3,-1.0,6.296471e-08,1.256659e-07,5.082828e-07,1.688218,0.000003,0.336888,4.968475e-09,2.958401,0.000002,...,-0.294201,0.310086,0.390273,-0.323726,0.029401,-0.503890,5.284086e-11,a,10,1
4,-1.0,1.348794e-07,2.556185e-07,1.272476e-06,1.709531,0.000007,0.410434,1.265314e-08,3.100358,0.000003,...,-0.205999,-0.422331,-0.518193,-0.079592,-0.380073,0.130950,1.105781e-10,a,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,-1.0,1.198905e-07,1.962642e-07,5.534923e-07,1.570978,0.000002,0.179934,4.407703e-09,9.176377,0.000002,...,-0.381015,0.000000,0.296897,-0.208100,-0.045185,-0.152193,1.272184e+00,h,6,20
2696,-1.0,8.276883e-08,1.388573e-07,1.346255e-06,1.782040,0.000007,0.297171,1.069583e-08,8.412500,0.000003,...,0.008490,0.296897,0.000000,0.102345,0.523751,-0.140699,1.782883e+00,h,6,20
2697,-1.0,1.159373e-07,1.994772e-07,1.033203e-06,1.673019,0.000004,0.298448,8.067409e-09,3.883632,0.000002,...,0.521226,-0.208100,0.102345,0.000000,0.108494,0.660035,3.140469e+00,h,6,20
2698,-1.0,3.440238e-08,9.217395e-08,1.591280e-06,1.887136,0.000008,0.516783,1.256575e-08,3.034146,0.000003,...,-0.200788,-0.045185,0.523751,0.108494,0.000000,0.144154,3.688495e+00,h,6,20


In [ ]:
n_subjects = 21
n_films = 8
n_channels = 18
channel_names = ['Fz', 'F3', 'F7','C3', 'T7', 'Pz', 'P3','P7', 'O1', 'Oz', 'O2','P4', 'P8', 'Cz','C4', 'T8', 'F4', 'F8']
for Subject in tqdm_notebook(range(1, n_subjects + 1)):
    if Subject == 5 or Subject == 21:
        continue
    for film in ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']:
        if not os.path.isfile('cropped_new/{}/watch_{}.fif'.format(Subject, film)):
            continue

        # читаем просмотр из файла
        
        watch = mne.io.read_raw('cropped_new/{}/watch_{}.fif'.format(Subject, film))
        relax = mne.io.read_raw()
        #удаляем ненужные каналы
        
        if 'VEOG' in raw.ch_names:
            raw.drop_channels(['VEOG'])
        raw.drop_channels(['Pulse', 'Zygom', 'Corr', 'EDA', 'Mark'])
        
        for channel in range(n_channels):
#            if os.path.isfile('features_neurokit/S_{}_F_{}_{}_.xlsx'.format(Subject, film, channel_names[channel])):
#                continue
            df, info = nk.complexity(raw[channel_names[channel]][0][0], which = ['fast', 'medium'])
            df['Subj'] =  Subject
            df['film'] = ord(film) - 96
            df['ch'] = channel + 1
            df['labels'] = ratings.loc[Subject - 1][film + '.mp4']
            df.to_excel('features_neurokit_medium/S_{}_F_{}_{}_.xlsx'.format(Subject, film, channel_names[channel]), index=False)

In [10]:
n_subjects = 21
n_films = 8
n_channels = 18
channel_names = ['Fz', 'F3', 'F7','C3', 'T7', 'Pz', 'P3','P7', 'O1', 'Oz', 'O2','P4', 'P8', 'Cz','C4', 'T8', 'F4', 'F8']
for Subject in tqdm_notebook(range(1, n_subjects + 1)):
    filtered = mne.io.read_raw('raw_filtered_fif/S{}_filtered_raw.fif'.format('0' + str(Subject) if Subject < 10 else Subject))
    raw = mne.io.read_raw('raw_full/' + (('00' + str(Subject) if Subject < 10 else '0' + str(Subject)) + 'a' if Subject == 2 or Subject == 5 else ('00' + str(Subject) if Subject < 10 else '0' + str(Subject))) + '.vhdr')
    annot = filtered.annotations
    n_events = 12 if Subject == 11 else 16
    for i in range(0, n_events, 2):
        film = annot[i + 1]['description'].split('_')[-1]
        if os.path.isfile('features_bio_new/S_{}_F_{}_{}_ppg.xlsx'.format(Subject, film, 'O2')):
            continue
        try:
            watch = raw.copy().crop(annot[i + 1]['onset'], annot[i + 1]['onset'] + annot[i + 1]['duration'])
        except Exception as e:
            watch = raw.copy().crop(annot[i + 1]['onset'], annot[i + 1]['onset'] + annot[i + 1]['duration'] - 0.01)
            
        relax = raw.copy().crop(annot[i]['onset'], annot[i]['onset'] + annot[i]['duration'])
        cleaned_relax = nk.ppg_clean(relax['Pulse'][0][0], sampling_rate=1000)
        info_ppg_relax = nk.ppg_findpeaks(cleaned_relax)
        cleaned_watch = nk.ppg_clean(watch['Pulse'][0][0], sampling_rate=1000)
        info_ppg_watch = nk.ppg_findpeaks(cleaned_watch)
        ppg_watch, info_ppg_watch = nk.ppg_process(watch['Pulse'][0][0], sampling_rate=watch.info['sfreq'])
        analyze_ppg_watch = nk.ppg_analyze(ppg_watch, sampling_rate=watch.info['sfreq'])
        
        eda_relax, info_eda_relax = nk.eda_process(relax['EDA'][0][0], sampling_rate=watch.info['sfreq'])
        analyze_eda_relax = nk.eda_analyze(eda_relax, sampling_rate=watch.info['sfreq'])
        eda_watch, info_eda_watch = nk.eda_process(watch['EDA'][0][0], sampling_rate=watch.info['sfreq'])
        analyze_eda_watch = nk.eda_analyze(eda_watch, sampling_rate=watch.info['sfreq'])
        emg_relax, info_emg_relax = nk.emg_process(relax['Zygom'][0][0], sampling_rate=watch.info['sfreq'])
        analyze_emg_relax = nk.emg_analyze(emg_relax, sampling_rate=watch.info['sfreq'])
        emg_watch, info_emg_watch = nk.emg_process(watch['Zygom'][0][0], sampling_rate=watch.info['sfreq'])
        analyze_emg_watch = nk.emg_analyze(emg_watch, sampling_rate=watch.info['sfreq'])
        df = pd.DataFrame({
            'ECG_Peaks_N_watch' : len(info_ppg_watch['PPG_Peaks']),
            'ECG_Peaks_N_relax' : len(info_ppg_relax['PPG_Peaks']),
            'SCR_Peaks_N_watch' : analyze_eda_watch['SCR_Peaks_N'],
            'SCR_Peaks_Amplitude_Mean_watch' : analyze_eda_watch['SCR_Peaks_Amplitude_Mean'],
            'SCR_Peaks_Amplitude_Mean_relax' : analyze_eda_relax['SCR_Peaks_Amplitude_Mean'],
            'SCR_Peaks_Amplitude_Mean_watch-relax' : analyze_eda_watch['SCR_Peaks_Amplitude_Mean'] - analyze_eda_relax['SCR_Peaks_Amplitude_Mean'],
            'EMG_Activation_N_watch' : analyze_emg_watch['EMG_Activation_N'],
            'EMG_Amplitude_Mean_watch': analyze_emg_watch['EMG_Amplitude_Mean'],
            
        })
        df = pd.concat([analyze_ppg_watch, analyze_eda_watch, analyze_emg_watch], axis=0)

        for channel in range(n_channels):
            df.to_excel('features_bio_new/S_{}_F_{}_{}_ppg.xlsx'.format(Subject, film, channel_names[channel]), index=False)

  0%|          | 0/21 [00:00<?, ?it/s]

In [ ]:
!pip install 

In [13]:
n_subjects = 21
n_films = 8
n_channels = 18
channel_names = ['Fz', 'F3', 'F7','C3', 'T7', 'Pz', 'P3','P7', 'O1', 'Oz', 'O2','P4', 'P8', 'Cz','C4', 'T8', 'F4', 'F8']
for Subject in tqdm_notebook(range(4, 5)):
    frames = []
    frames_rel = []
    if Subject == 5:
        continue
    for film in ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']:
        if not os.path.isfile('cropped_new/{}/watch_{}.fif'.format(Subject, film)):
            continue
        relax = mne.io.read_raw('cropped_new/{}/relax_{}.fif'.format(Subject, film))
        watch = mne.io.read_raw('cropped_new/{}/watch_{}.fif'.format(Subject, film))
        sf = relax.info['sfreq']
        win = int(sf)
        
        if 'VEOG' in relax.ch_names:
            relax.drop_channels(['VEOG'])
            watch.drop_channels(['VEOG'])
        relax.drop_channels(['Pulse', 'Zygom', 'Corr', 'EDA', 'Mark'])
        watch.drop_channels(['Pulse', 'Zygom', 'Corr', 'EDA', 'Mark'])
        

        freqs_relax, psd_relax = welch(relax.get_data() * 1e6, sf, nperseg=win, average='median')
        freqs_watch, psd_watch = welch(watch.get_data() * 1e6, sf, nperseg=win, average='median')
        
        bp_relax = yasa.bandpower_from_psd(psd_relax, freqs_relax, relax.ch_names, bands=
                                   [(4, 8, 'Theta'), (8, 12, 'Alpha'), (12, 30, 'Beta')], relative=False).drop(columns=['FreqRes', 'Relative', 'TotalAbsPow'])
        bp_watch = yasa.bandpower_from_psd(psd_watch, freqs_watch, watch.ch_names, bands=
                                   [(4, 8, 'Theta'), (8, 12, 'Alpha'), (12, 30, 'Beta')], relative=False).drop(columns=['FreqRes', 'Relative', 'TotalAbsPow'])
        bp_ratio_relax = yasa.bandpower(relax, relax.info['sfreq'], bands=
                                   [(4, 8, 'Theta_ratio'), (8, 12, 'Alpha_ratio'), (12, 30, 'Beta_ratio')], relative=True).drop(columns=['FreqRes', 'Relative'])
        bp_ratio_watch = yasa.bandpower(watch, watch.info['sfreq'], bands=
                                   [(4, 8, 'Theta_ratio'), (8, 12, 'Alpha_ratio'), (12, 30, 'Beta_ratio')], relative=True).drop(columns=['FreqRes', 'Relative'])
        for channel in range(n_channels):
            df_relax = pd.DataFrame(bp_relax.loc[channel]).transpose()
            df_watch = pd.DataFrame(bp_watch.loc[channel]).transpose()
            dr = pd.DataFrame(bp_ratio_relax.loc[relax.ch_names[channel]]).transpose()
            dr.rename(index={relax.ch_names[channel]:channel}, inplace=True)
            df_relax = df_relax.join(dr)
            dw = pd.DataFrame(bp_ratio_watch.loc[watch.ch_names[channel]]).transpose()
            dw.rename(index={watch.ch_names[channel]:channel}, inplace=True)
            df_watch = df_watch.join(dw)
            df_relax.rename(index={channel:'F' + str(film) + '_relax'}, inplace=True)
            df_watch.rename(index={channel:'F' + str(film) + '_watch'}, inplace=True)

            df_rel = {
                'Theta_watch-relax' : float(df_watch['Theta']) - float(df_relax['Theta']),
                'Alpha_watch-relax' : float(df_watch['Alpha']) - float(df_relax['Alpha']),
                'Beta_watch-relax' : float(df_watch['Beta']) - float(df_relax['Beta']),
            }
            df_rel = pd.DataFrame(df_rel, index=['F' + str(film) + '_' + watch.ch_names[channel]])
            df_div = {
                'Beta/Alpha' : float(df_rel['Beta_watch-relax']) / float(df_rel['Alpha_watch-relax']),
                'Beta/(Alpha + Theta)' : float(df_rel['Beta_watch-relax']) / (float(df_rel['Alpha_watch-relax'] + float(df_rel['Theta_watch-relax']))),
                'Beta_watch/Alpha_watch' : float(df_watch['Beta']) / float(df_watch['Alpha']),
                'Beta_watch/(Alpha_watch + Theta_watch)' : float(df_watch['Beta']) / (float(df_watch['Alpha'] + float(df_watch['Theta']))),
                'Beta_watch/Alpha_watch - Beta_relax/Alpha_relax' : float(df_watch['Beta']) / float(df_watch['Alpha']) - float(df_relax['Beta']) / float(df_relax['Alpha']),
                'Beta_watch/(Alpha_watch + Theta_relax) - Beta_relax/(Alpha_relax + Theta_relax)' : float(df_watch['Beta']) / (float(df_watch['Alpha'] + float(df_watch['Theta']))) - float(df_relax['Beta']) / (float(df_relax['Alpha'] + float(df_relax['Theta'])))
            }
            df_div = pd.DataFrame(df_div, index=['F' + str(film) + '_' + watch.ch_names[channel]])
            df_all = df_rel.join(df_div)
            frames_rel.append(df_all)
            frames.append(df_relax)
            frames.append(df_watch)
    result_rel = pd.concat(frames_rel)
    result_rel.to_excel('features_yasa_new_2/S' + str(Subject) + '_ratio.xlsx')
    result = pd.concat(frames)
    result.to_excel('features_yasa_new_2/S' + str(Subject) + '.xlsx')

  0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
n_subjects = 21
n_films = 8
n_channels = 18
ind = 0
ch_names = ['Fz', 'F3', 'F7','C3', 'T7', 'Pz', 'P3','P7', 'O1', 'Oz', 'O2','P4', 'P8', 'Cz','C4', 'T8', 'F4', 'F8']
data_all = []
for Subject in tqdm_notebook(range(4, 5)):
    if Subject == 5:
        continue
    df_rel = pd.read_excel('features_yasa_new_2/S{}_ratio.xlsx'.format(Subject))
    df = pd.read_excel('features_yasa_new_2/S{}.xlsx'.format(Subject))
    first = 0
    second = 1
    for film in ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']:
        for channel in range(n_channels):

            dp = df_rel.loc[(df_rel['Unnamed: 0'] == 'F' + str(film) + '_' + ch_names[channel])].rename(index={first:ind})
            d = df.loc[(df['Chan'] == ch_names[channel]) & (df['Unnamed: 0'] == 'F' + str(film) + '_watch')].rename(index={second:ind})
            if dp.empty or d.empty:
                print('{}{}{}'.format(Subject, film, ch_names[channel]))
                continue
            d['Theta_watch-relax'] = float(dp['Theta_watch-relax'])
            d['Alpha_watch-relax'] = float(dp['Alpha_watch-relax'])
            d['Beta_watch-relax'] = float(dp['Beta_watch-relax'])
            d['Beta/Alpha'] = float(dp['Beta/Alpha'])
            d['Beta/(Alpha + Theta)'] = float(dp['Beta/(Alpha + Theta)'])
            d['Beta_watch/Alpha_watch'] = float(dp['Beta_watch/Alpha_watch'])
            d['Beta_watch/(Alpha_watch + Theta_watch)'] = float(dp['Beta_watch/(Alpha_watch + Theta_watch)'])
            d['Beta_watch/Alpha_watch - Beta_relax/Alpha_relax'] = float(dp['Beta_watch/Alpha_watch - Beta_relax/Alpha_relax'])
            d['Beta_watch/(Alpha_watch + Theta_relax) - Beta_relax/(Alpha_relax + Theta_relax)'] = float(dp['Beta_watch/(Alpha_watch + Theta_relax) - Beta_relax/(Alpha_relax + Theta_relax)'])
            d = d.drop(['Unnamed: 0', 'Chan'], axis=1)
            d.to_excel('features_spectral_new/S{}_F_{}_{}_.xlsx'.format(Subject, film, ch_names[channel]), index=False)
            d['Subj'] =  Subject
            d['film'] = film
            d['ch'] = ch_names[channel]
            d['labels'] = ratings.loc[Subject - 1][film + '.mp4']
            data_all.append(d)
            first += 1
            second += 2
            ind += 1

  0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
data_all = pd.concat(data_all)
data_all

,Theta,Alpha,Beta,Theta_ratio,Alpha_ratio,Beta_ratio,TotalAbsPow,Theta_watch-relax,Alpha_watch-relax,Beta_watch-relax,Beta/Alpha,Beta/(Alpha + Theta),Beta_watch/Alpha_watch,Beta_watch/(Alpha_watch + Theta_watch),Beta_watch/Alpha_watch - Beta_relax/Alpha_relax,Beta_watch/(Alpha_watch + Theta_relax) - Beta_relax/(Alpha_relax + Theta_relax),Subj,film,ch,labels
0,2.401640,2.783825,2.914699,0.318677,0.340933,0.340389,8.088658,-1.426278,0.805397,-0.424500,-0.527069,0.683705,1.047013,0.562090,-0.640792,-0.013004,4,a,Fz,8.0
1,1.848014,4.860342,3.861837,0.180087,0.491270,0.328643,10.238296,-0.034008,1.992814,-0.008136,-0.004083,-0.004153,0.794561,0.575676,-0.555024,-0.239133,4,a,F3,8.0
2,5.193237,4.188857,5.373905,0.372848,0.289807,0.337345,15.082173,-2.212639,-0.432822,-1.033615,2.388083,0.390713,1.282905,0.572783,-0.103500,0.040046,4,a,F7,8.0
3,3.109635,6.614255,5.331608,0.211328,0.455727,0.332945,14.337754,0.044150,2.854654,-0.113623,-0.039803,-0.039196,0.806078,0.548300,-0.642275,-0.249526,4,a,C3,8.0
4,3.711169,2.691938,3.848378,0.370692,0.267805,0.361503,10.278948,-1.303629,-0.368502,-0.990704,2.688463,0.592480,1.429594,0.601017,-0.151578,0.001768,4,a,T7,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,3.575201,1.851132,3.678716,0.387283,0.206470,0.406246,9.303020,0.170065,-0.803866,-1.172820,1.458974,1.850454,1.987280,0.677938,0.159958,-0.122628,4,h,Cz,6.0
140,4.967528,4.148056,6.345562,0.318008,0.279010,0.402982,15.554973,1.112179,-0.495866,-0.648755,1.308326,-1.052640,1.529768,0.696122,0.023645,-0.126809,4,h,C4,6.0
141,6.636565,5.209866,10.797049,0.290776,0.230819,0.478405,23.069312,1.583170,-0.458713,-4.950189,10.791478,-4.402292,2.072424,0.911418,-0.705563,-0.557270,4,h,T8,6.0
142,4.556124,3.119000,6.451290,0.322952,0.221992,0.455056,14.506383,0.565723,0.190684,-1.620096,-8.496252,-2.141831,2.068384,0.840545,-0.687939,-0.326056,4,h,F4,6.0


In [17]:
data_all.to_excel('Spectral.xlsx', index=False)

In [25]:
ch_names = ['Fz', 'F3', 'F7','C3', 'T7', 'Pz', 'P3','P7', 'O1', 'Oz', 'O2','P4', 'P8', 'Cz','C4', 'T8', 'F4', 'F8']
features_all = pd.DataFrame()
n_subjects = 20
n_films = 8
n_channels = 18
for Subject in tqdm_notebook(range(1, n_subjects + 1)):
    if Subject == 5:
        continue
    for film in ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']:
        for channel in range(n_channels):
            if not os.path.isfile('features_spectral_new/S{}_F_{}_{}_.xlsx'.format(int(Subject), film, ch_names[channel])):
                continue
            yasa = pd.read_excel('features_spectral_new/S{}_F_{}_{}_.xlsx'.format(int(Subject), film, ch_names[channel]))
            neurokit = pd.read_excel('features_neurokit_new/S_{}_F_{}_{}_.xlsx'.format(int(Subject), film, ch_names[channel]), engine='openpyxl')
            #bio = pd.read_excel('features_bio_new/S_{}_F_{}_{}_ppg.xlsx'.format(Subject, film, channel_names[channel]))
            df = pd.concat([yasa, neurokit], axis=1)
            features_all = pd.concat([features_all, df], axis=0)

  0%|          | 0/20 [00:00<?, ?it/s]

In [26]:
features_all

,Theta,Alpha,Beta,Theta_ratio,Alpha_ratio,Beta_ratio,TotalAbsPow,Theta_watch-relax,Alpha_watch-relax,Beta_watch-relax,...,PFD,RR,SFD,SVDEn,ShanEn,SpEn,Subj,film,ch,labels
0,5.478965,2.526810,2.764925,0.507505,0.234057,0.258438,10.995244,0.459282,0.486312,0.426391,...,1.003339,0.075773,1.608428,0.535017,15.271892,0.630001,1,1,1,10
0,2.884534,1.913928,2.768027,0.390887,0.252898,0.356215,7.767308,-0.600457,0.504085,0.141512,...,1.003882,0.108728,1.616296,0.592474,15.271639,0.690453,1,1,2,10
0,7.166805,4.005514,7.978071,0.382585,0.212711,0.404704,20.006198,0.683279,0.348539,2.843236,...,1.003305,0.085986,1.608730,0.554813,15.271892,0.578646,1,1,3,10
0,2.694183,2.027418,3.852692,0.318860,0.240955,0.440185,8.883501,0.197800,0.543780,1.163700,...,1.003822,0.113499,1.623063,0.599554,15.271892,0.585646,1,1,4,10
0,6.281124,4.022892,8.421597,0.332227,0.218676,0.449097,19.067099,1.717636,0.629256,3.446314,...,1.004642,0.168478,1.645645,0.665271,15.272094,0.680171,1,1,5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,4.819919,2.735769,3.480329,0.445228,0.241847,0.312926,11.573306,0.146283,0.010476,-0.631298,...,1.002190,0.032390,1.538414,0.412120,15.571695,0.525844,20,8,14,6
0,3.433117,1.952647,7.195257,0.273939,0.158074,0.567987,12.843455,0.290486,-0.188057,2.239323,...,1.003640,0.088322,1.600959,0.559143,15.571695,0.522496,20,8,15,6
0,5.010104,2.711312,6.274102,0.362032,0.195539,0.442428,14.477907,1.689780,-0.323697,1.344576,...,1.003595,0.089069,1.615041,0.560490,15.571900,0.621396,20,8,16,6
0,1.430647,1.241856,8.130359,0.137981,0.108538,0.753481,10.736331,-0.087192,0.081011,3.753380,...,1.006014,0.267146,1.682560,0.743279,15.571695,0.614844,20,8,17,6


In [29]:
features_all.to_excel('features_neurokit2_new2.xlsx', index=False)

In [28]:
features_all['ECG_Rate_Mean_watch']

KeyError: 'ECG_Rate_Mean_watch'

In [25]:
features_all.to_csv('matrix_bio3.csv', index=False)

In [14]:
features_all.to_csv('matrix.csv', index=False)

In [12]:
features_all = features_all.dropna(axis=1)

In [13]:
features_all.to_csv('matrix_bio.csv', index=False)
features_all

,Theta,Alpha,Beta,Theta_ratio,Alpha_ratio,Beta_ratio,TotalAbsPow,Theta_watch-relax,Alpha_watch-relax,Beta_watch-relax,...,Subj,film,ch,labels,ECG_Rate_Mean_watch,SCR_Peaks_N_watch,SCR_Peaks_Amplitude_Mean_watch,SCR_Peaks_Amplitude_Mean_relax,SCR_Peaks_Amplitude_Mean_watch-relax,EMG_Activation_N_watch
0,13.011456,5.120684,6.871645,0.518855,0.206702,0.274443,25.179888,3.734071,2.042683,2.376408,...,1,1,1,10,64.891654,20,0.000148,0.000012,0.000136,205
0,6.563671,3.319401,5.831020,0.416457,0.212868,0.370676,15.653934,0.458212,1.255145,1.792163,...,1,1,2,10,64.891654,20,0.000148,0.000012,0.000136,205
0,4.540355,2.501262,6.033912,0.340833,0.193027,0.466140,12.963027,-1.882338,0.240152,2.327508,...,1,1,3,10,64.891654,20,0.000148,0.000012,0.000136,205
0,9.124489,5.293534,7.495714,0.411476,0.247264,0.341260,21.925581,3.279015,1.941784,2.923710,...,1,1,4,10,64.891654,20,0.000148,0.000012,0.000136,205
0,3.749049,3.162736,92.597395,0.037305,0.032126,0.930570,99.696791,0.333485,0.801545,90.107741,...,1,1,5,10,64.891654,20,0.000148,0.000012,0.000136,205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,21.193541,4.736696,11.184029,0.568394,0.131438,0.300168,37.174902,12.163021,-3.264240,-1.078549,...,21,8,14,1,14.981789,20,0.000001,0.000004,-0.000003,0
0,17.104650,3.568935,8.170350,0.593663,0.123477,0.282860,28.988928,10.446148,-2.107908,-1.032185,...,21,8,15,1,14.981789,20,0.000001,0.000004,-0.000003,0
0,7.015250,1.543985,5.893248,0.484762,0.106493,0.408745,14.441618,3.185019,-1.219592,-0.234495,...,21,8,16,1,14.981789,20,0.000001,0.000004,-0.000003,0
0,12.301713,3.111477,13.173423,0.430884,0.108658,0.460458,28.584464,7.211230,-0.653889,3.956880,...,21,8,17,1,14.981789,20,0.000001,0.000004,-0.000003,0


In [ ]:
n_subjects = 21
matrix = pd.DataFrame()
for Subject in tqdm_notebook(range(1, n_subjects + 1)):
    if Subject == 21 or Subject == 5:
        continue
    filtered = mne.io.read_raw('raw_filtered_fif/S{}_filtered_raw.fif'.format('0' + str(Subject) if Subject < 10 else Subject))
    raw = mne.io.read_raw('raw_full/' + (('00' + str(Subject) if Subject < 10 else '0' + str(Subject)) + 'a' if Subject == 2 or Subject == 5 else ('00' + str(Subject) if Subject < 10 else '0' + str(Subject))) + '.vhdr')
    annot = filtered.annotations
    n_events = 12 if Subject == 11 else 16
    for i in range(0, n_events, 2):
        film = annot[i + 1]['description'].split('_')[-1]
        try:
            watch = raw.copy().crop(annot[i + 1]['onset'], annot[i + 1]['onset'] + annot[i + 1]['duration'])
        except Exception as e:
            watch = raw.copy().crop(annot[i + 1]['onset'], annot[i + 1]['onset'] + annot[i + 1]['duration'] - 0.01)